In [23]:
import random, csv, time, os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from scipy import optimize
from scipy.stats import bernoulli

%matplotlib inline
# Videos result files are from http://sunai.uoc.edu/traits/layout2/results/

In [2]:
global pairs_truth, video_score, video_num, total_pairs, w_hat, test_pairs

In [123]:
video_num = 50

video_score = np.random.uniform(-5,5,video_num)
resolution = 0.1
video_score = np.round(video_score/resolution)*resolution

pairs_truth = []

for i in range(len(video_score)-1):
    for j in range(i+1, len(video_score)):
        if video_score[i] > video_score[j]:
            pairs_truth.append((i,j))
        else:
            pairs_truth.append((j,i))

total_pairs = len(pairs_truth)  

In [124]:
def append_pair(comp, name1, name2, name_to_num):
    if comp == '-1':
        return((name_to_num[name2], name_to_num[name1]))
    if comp == '1':
        return((name_to_num[name1], name_to_num[name2]))
    
def calc_error(prediction, truth):
    return np.linalg.norm(prediction - truth)

def mle(w, pairs):    
    """
    Calculate the MLE of w.
    Due to the fact that optimize.minimize only takes 1 input, 
    here the pair information is imported from global variable
    """    
    out = 1
                
    for pair in pairs:
        out *= 1/(1+np.exp(-w[pair[0]] + w[pair[1]]))   
        
    return -np.log(out)

def gradient(w,pairs):
    grad = []
    for i in range(len(w)):
        gradient = 0

        for pair in pairs:
            if i == pair[0]:
                out = -1
            elif i == pair[1]:
                out = 1  
            else:
                continue
            gradient -= out / (1/(np.exp(w[pair[1]]-w[pair[0]])) +1 )

        grad.append(-gradient)
        
    return np.array(grad)

def compare_rank(video_score, results, verbose=False, hist=False):
    true_order = np.array(video_score).argsort()
    true_ranks = true_order.argsort()

    temp_o = np.array(results).argsort()
    temp_r = temp_o.argsort()

    resolution = 0.1
    video_score_results = np.round(np.array(results)/resolution)*resolution
    
    if verbose:
        print 'Result Order \t True Order \t Result Score \t Ture Score'
        for i in range(len(temp_r)):
            print temp_r[i], '\t\t', true_ranks[i], '\t\t', video_score_results[i], '\t\t', video_score[i]
            
    if hist:
        diff = np.abs(temp_r - true_ranks)
        plt.hist(diff, alpha=0.5)

# Below is just a general test to see how if the model performs nicely.

In [ ]:
w = np.ones(video_num)
# test_pairs = pairs_truth
test_pairs = [pairs_truth[i] for i in random.sample(range(total_pairs), int(total_pairs*1))]
print len(test_pairs)
start_time = time.time()

res = optimize.minimize(mle, w, 
                        method='Newton-CG',
                        jac=gradient, 
                        args=(test_pairs,),
                        options={'disp': True})

print  'Time Spent: %.2f seconds' %float(time.time() - start_time)

1225


In [ ]:
compare_rank(video_score, res.x, verbose=False, hist=True)

# Introducing different methods of calculating error.

In [89]:
def matching_func(param, video_score, w_hat):
    """
    Function used to calculate L2 norm of v and w_star. Used to find a and b.
    Here param is [a, b]
    """
    return np.linalg.norm(video_score - param[0]*np.array(w_hat) - param[1])


def regularized_vector(w_hat):
    """
    Function to generate vector v after using matching_func to find optimal a and b
    """
    coeff = optimize.minimize(matching_func, [0, 0], args=(video_score, w_hat))

    a = coeff['x'][0]
    b = coeff['x'][1]
    v = a*np.array(w_hat)+b
    return v

def performance_isabelle(video_score, video_num, w_hat):
    """
    Performance evaluation using method proposed by Isabelle. 
    """
    epsilon = 0.0001
    error = 0
    v = regularized_vector(w_hat)

    true_order = np.array(video_score).argsort()

    what = [ w_hat[i] for i in true_order]
    wstar = [ video_score[i] for i in true_order]

    for i in range(video_num-2):
        error += np.abs((wstar[i+1]-wstar[i])/(wstar[i+2]-wstar[i]+epsilon)-
                        (what[i+1]-what[i])/(what[i+2]-what[i]+epsilon))
    
    return error

def performance_nihar(video_score, w_hat):
    """
    Performance evaluation using method proposed by Nihar
    """
    v = regularized_vector(w_hat)
    return np.linalg.norm(video_score - v)


def performance_rank_compare(video_score, w_hat, pairs):
    """
    Using the basics rank compare method
    """
    true_order = np.array(video_score).argsort()
    true_ranks = true_order.argsort()

    temp_o = np.array(w_hat).argsort()
    temp_r = temp_o.argsort()
    
    true_scores = np.zeros(len(w_hat))
    temp_scores = np.zeros(len(w_hat))
    
    for i in range(len(scores)):
        true_scores[i] = 1 - true_ranks[i]/len(scores)
        counter = 0
        for pair in pairs:
            if pair[0] == i:
                temp_scores[i] += 1
                counter += 1
            elif pair[1] == i:
                counter += 1
        temp_scores[i] /= counter
    return 

### Sparsity

In [101]:
results = {}
num_test_pairs = (np.array((1,0.9,0.8,0.7,0.6,0.5,0.4,0.3,0.2,0.1))* total_pairs).astype(int)
# num_test_pairs = (np.array((1,0.8,0.6,0.4,0.2))* total_pairs).astype(int)
# num_test_pairs = (np.array((0.4,0.2))* total_pairs).astype(int)


sparsity_error_nihar = []
sparsity_error_isabelle = []

for test_pair_num in num_test_pairs:
    
    print 'Current evaluating with %d test pairs' % test_pair_num
    
    test_pairs = [pairs_truth[i] for i in random.sample(range(total_pairs), test_pair_num)]

    w = np.ones(video_num)

    res = optimize.minimize(mle, w, 
                            method='Newton-CG',
                            jac=gradient, 
                            args=(test_pairs,),
                            options={'disp': False})
    
    w_hat = res.x
    results[str(test_pair_num)] = w_hat
#     compare_rank(video_score, res.x, verbose=False, hist=False)
    
    sparsity_error_nihar.append(performance_nihar(video_score,w_hat))
    sparsity_error_isabelle.append(performance_isabelle(video_score, video_num, w_hat))

Current evaluating with 44850 test pairs
Current evaluating with 40365 test pairs


KeyboardInterrupt: 

In [ ]:
plt.figure()
plt.plot(sparsity_error_isabelle)
plt.figure()
plt.plot(sparsity_error_nihar)

### Different Number of Videos

In [ ]:
video_nums = [50,100,150,200,250,300]

Nihar = True
Isabelle = True
showResult = False

number_error_nihar = []
number_error_isabelle = []

for num in video_nums:
    
    video_num = num
    
    video_score = np.random.uniform(-5,5,video_num)
    resolution = 0.1
    video_score = np.round(video_score/resolution)*resolution

    pairs_truth = []

    for i in range(len(video_score)-1):
        for j in range(i+1, len(video_score)):
            if video_score[i] > video_score[j]:
                pairs_truth.append((i,j))
            else:
                pairs_truth.append((j,i))

    total_pairs = len(pairs_truth)  
    
    print 'Current evaluating with %d videos' % video_num
    
    test_pairs = pairs_truth
#     test_pairs = [pairs_truth[i] for i in random.sample(range(total_pairs), test_pair_num)]

    w = np.ones(video_num)

    res = optimize.minimize(mle, w, 
                            method='Newton-CG',
                            jac=gradient, 
                            args=(test_pairs,),
                            options={'disp': True})
    w_hat = res.x
    number_error_nihar.append(performance_nihar(video_score,w_hat))
    number_error_isabelle.append(performance_isabelle(video_score, video_num, w_hat))

### Generate data with BTL with diff sparsity

In [84]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def noise_generation(video_score, pair):
    """
    Generate a decision of whether to flip, using proability from normal distribution and then 
    Bernoulli to decide whether to flip
    """

    difference = video_score[pair[0]] - video_score[pair[1]]
    p = sigmoid(difference)*(1-sigmoid(difference))
    decision = bernoulli.rvs(p,size=1)

    return decision

In [ ]:
# num_test_pairs = (np.array((1,0.9,0.8,0.7,0.6,0.5,0.4,0.3,0.2,0.1))* total_pairs).astype(int)
num_test_pairs = (np.array((0.4,0.2))* total_pairs).astype(int)

sparsity_error_generative_nihar = []
sparsity_error_generative_isabelle = []

for test_pair_num in num_test_pairs:
    
    print 'Current evaluating with %d test pairs' % test_pair_num
    
    test_pairs = [pairs_truth[i] for i in random.sample(range(total_pairs), test_pair_num)]
    
    flip_count = 0
    for i in range(len(test_pairs)):
        if noise_generation(video_score, test_pairs[i]):
            temp = (test_pairs[i][1], test_pairs[i][0])
            test_pairs[i] = temp
            flip_count +=1
    

    w = np.ones(video_num)
    print 'evaluating', flip_count/len(test_pairs)
    res = optimize.minimize(mle, w, 
                            method='Newton-CG',
                            jac=gradient, 
                            args=(test_pairs,),
                            options={'disp': False})
    
    w_hat = res.x
    
    sparsity_error_generative_nihar.append(performance_nihar(video_score,w_hat))
    sparsity_error_generative_isabelle.append(performance_isabelle(video_score, video_num, w_hat))